In [6]:
import json
from tokenizers import ByteLevelBPETokenizer
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
with open(r'../data/aneks/processed/aneks.json', 'r') as f:
    aneks = json.load(f)

In [21]:
len(aneks)

516330

In [6]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(
    r'../data/aneks/processed/aneks.txt',
    vocab_size=16000,
    special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]", "[BOS]", "[EOS]"]
)

tokenizer.save(r'../data/tokenizers/gpt/tokenizer_gpt.json')

In [7]:
tokenizer.save(r'../data/tokenizers/gpt/tokenizer_gpt.json')

In [9]:
df = pd.DataFrame([v for v in aneks.values()]).rename({0:'anek'}, axis=1)

In [10]:
df

,anek
0,Выборы президентов США как будто сняты по реко...
1,"Если Китай для стран бывшего СССР это ""сын мам..."
2,Многие мужчины путают женитьбу с усыновлением
3,Жизненный парадокс: пока живешь в неблагополуч...
4,"Я помню доллар за 6000 рублей, так что знаю - ..."
...,...
528985,"Решила таганская братва отдохнуть, рыбки полов..."
528986,Попросила сестра брата-наркомана с ребенком по...
528987,"Жираф ежу рассказывает:\n - Эх, как же я люблю..."
528988,"Малыш : Карлсон, ты за кого голосовать-то буде..."


In [1]:
from transformers import PreTrainedTokenizerFast, AutoTokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained(r'D:\workspace\projects\anek\data\tokenizers\gpt_lora/')

In [11]:
tokenizer = PreTrainedTokenizerFast(
            tokenizer_file=r'D:\workspace\projects\anek\data\tokenizers\gpt\tokenizer_gpt.json',
            unk_token="[UNK]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            sep_token="[SEP]",
            mask_token="[MASK]",
            bos_token='[BOS]',
            eos_token='[EOS]'
        )

In [4]:
tokenizer.vocab_size

50257

In [11]:
df['ln'] = df.anek.apply(lambda x: len(tokenizer.tokenize(x)))

Token indices sequence length is longer than the specified maximum sequence length for this model (4009 > 2048). Running this sequence through the model will result in indexing errors


In [13]:
df.head()

,anek,ln
0,Выборы президентов США как будто сняты по реко...,43
1,"Если Китай для стран бывшего СССР это ""сын мам...",83
2,Многие мужчины путают женитьбу с усыновлением,9
3,Жизненный парадокс: пока живешь в неблагополуч...,65
4,"Я помню доллар за 6000 рублей, так что знаю - ...",18


In [14]:
df[df.ln > 256]

,anek,ln
698,Представьте ситуацию: колл-центр телефонных мо...,261
905,"Короче, пригоняет клиент свою машину в автосер...",261
908,"Православный, мусульманин и атеист умирают и п...",265
936,Приходит как-то трезвый чёрт к пьяному мужику ...,740
1138,Падает метеорит в лес. На утро институт направ...,367
...,...,...
528815,По Нью-Йоркской 42-й авеню идет солидный мен и...,276
528844,"Бежит волк за зайцем, с'есть его хочет и на те...",270
528870,Поехали как то мужики на охоту. Приехали и нач...,309
528881,Нью-Йорк. Из окна небоскреба падает и разбивае...,294


In [15]:
df = df[df.ln <= 256]
df = df['anek']

In [16]:
train, test = train_test_split(df, train_size=0.985)

In [17]:
test

509662    Какое самое страшное ругательство у глистов?\n...
463167    Решает молодая пара пожениться, но перед этим ...
46612     Чтобы уничтожить автопром достаточно запретить...
98390     Купил 8К телевизор, но Пугачева пела с черно-б...
411444    Язык Си придумали для программистов, которым б...
                                ...                        
471295    Oтвет мусульман христианам всего мира: фильм "...
70237     На каждое наше мягкое требование – а не могли ...
431319    По данным социологического опроса:\n\n1. самой...
209219    Бывало, ляпнешь что-то не подумав, а потом дум...
235820    Жители Республики Беларусь совершенно не поним...
Name: anek, Length: 7863, dtype: object

In [18]:
train.reset_index(drop=True).to_json(r'../data/aneks/processed/for_lora/train.json')
test.reset_index(drop=True).to_json(r'../data/aneks/processed/for_lora/test.json')

In [19]:
train

291149    В Амстердаме открылся первый дом свидании для ...
262238    Очень хочу, чтобы судьба взяла меня за волосы ...
58957     К молодому красавцу проктологу на приём приход...
157853    Правительство вводит новую единицу измерения -...
486951    Сын милиционера:\n- Папа, а что ты мне принес ...
                                ...                        
220824    - Почему девушки носят мини-юбки?\n- Товар рек...
302955    - Обними меня!\n - Не, Люся, я боюсь - током с...
364286    Когда шведы узнали, что их кронпринцесса выход...
521880    Народная примета:\nЕсли пpогpаммист в 09.00 ут...
441940    Посещая свадьбы, Максим Горький отмечал "А у м...
Name: anek, Length: 516330, dtype: object